In [1]:
from ultralytics import YOLO
import os
import shutil
import random

In [2]:
# setup directory paths
dataset_dir = 'dataset/pose'
split_dir = 'datasets/split'
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')
train_images_dir = os.path.join(split_dir, 'train/images')
train_labels_dir = os.path.join(split_dir, 'train/labels')
val_images_dir = os.path.join(split_dir, 'val/images')
val_labels_dir = os.path.join(split_dir, 'val/labels')

In [3]:
# extract the name and replace .jpg with .txt so we can match the images with their respective labels
images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

split_ratio = 0.8
split_index = int(len(images) * split_ratio)

# do a little shuffling
combined = list(zip(images, labels))
random.shuffle(combined)
images[:], labels[:] = zip(*combined)

# split into training and validation data sets
train_images = images[:split_index]
train_labels = labels[:split_index]
val_images = images[split_index:]
val_labels = labels[split_index:]

In [4]:
# setup directories
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# empty out both folders, then copy the split
for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    for file in os.listdir(dir_path):
        file_path = os.path.join(train_images_dir, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f'Failed to delete {file}, exception: {e}')

# copy files
for image, label in zip(train_images, train_labels):
    shutil.copyfile(os.path.join(images_dir, image), os.path.join(train_images_dir, image))
    shutil.copyfile(os.path.join(labels_dir, label), os.path.join(train_labels_dir, label))
    
for image, label in zip(val_images, val_labels):
    shutil.copyfile(os.path.join(images_dir, image), os.path.join(val_images_dir, image))
    shutil.copyfile(os.path.join(labels_dir, label), os.path.join(val_labels_dir, label))

In [5]:
model = YOLO("yolov8m-pose.pt")

In [6]:
model.train(data="data.yaml", epochs=100, imgsz=640, batch=32)

New https://pypi.org/project/ultralytics/8.2.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.25 🚀 Python-3.11.6 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24214MiB)
engine/trainer: task=pose, mode=train, model=yolov8m-pose.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

/home/kroep/miniforge3/envs/bee/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608853085/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
train: Scanning /home/kroep/dev/projects/bees/bee_detection/datasets/split/train/labels... 320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 320/320 [00:00<00:00, 2486.47it/s]

train: New cache created: /home/kroep/dev/projects/bees/bee_detection/datasets/split/train/labels.cache
WARNING ⚠️ No 'flip_idx' array defined in data.yaml, setting augmentation 'fliplr=0.0'



val: Scanning /home/kroep/dev/projects/bees/bee_detection/datasets/split/val/labels... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<00:00, 1161.19it/s]

val: New cache created: /home/kroep/dev/projects/bees/bee_detection/datasets/split/val/labels.cache


...

                   all         80        842      0.965       0.95      0.981      0.788      0.965       0.95      0.981      0.971



100 epochs completed in 0.127 hours.
Optimizer stripped from runs/pose/train15/weights/last.pt, 53.2MB
Optimizer stripped from runs/pose/train15/weights/best.pt, 53.2MB

Validating runs/pose/train15/weights/best.pt...
Ultralytics YOLOv8.2.25 🚀 Python-3.11.6 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24214MiB)
YOLOv8m-pose summary (fused): 237 layers, 26401087 parameters, 0 gradients


/home/kroep/miniforge3/envs/bee/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608853085/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.96it/s]


                   all         80        842      0.972      0.938      0.981      0.788      0.971      0.942      0.983      0.972
Speed: 0.1ms preprocess, 2.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/pose/train15


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc7c6104a50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    

In [7]:
best_model = YOLO("runs/pose/train15/weights/best.pt")

In [8]:
results = best_model.val(save=False, plots=True, save_hybrid=True, conf=False, show_labels=False, show_boxes=False, verbose=False, show=True)

Ultralytics YOLOv8.2.25 🚀 Python-3.11.6 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24214MiB)
YOLOv8m-pose summary (fused): 237 layers, 26401087 parameters, 0 gradients


val: Scanning /home/kroep/dev/projects/bees/bee_detection/datasets/split/val/labels.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.49it/s]


                   all         80        842          1          1      0.995      0.995     0.0725     0.0819      0.101     0.0554
Speed: 1.1ms preprocess, 4.9ms inference, 0.0ms loss, 8.7ms postprocess per image
Results saved to runs/pose/val14


In [9]:
import cv2
from tqdm import tqdm

output_dir = 'output'
val_data_dir = "datasets/split/val/images/"

os.makedirs(output_dir, exist_ok=True)

for img_file in tqdm(os.listdir(val_data_dir)):
    img_path = os.path.join(val_data_dir, img_file)

    # Read the original image
    original_img = cv2.imread(img_path)

    # Get predictions
    results = best_model.predict(img_path, save=False, save_txt=False, show_labels=False, show_conf=False)

    # Get the image with predictions
    pred_img = results[0].plot(labels=False, show=False)

    # Concatenate original and prediction images side by side
    side_by_side = cv2.hconcat([original_img, pred_img])

    # Save the concatenated image
    output_path = os.path.join(output_dir, f'side_by_side_{img_file}')
    cv2.imwrite(output_path, side_by_side)

  0%|          | 0/80 [00:00<?, ?it/s]


image 1/1 /home/kroep/dev/projects/bees/bee_detection/datasets/split/val/images/20230609d402.jpg: 384x640 5 bees, 75.5ms
Speed: 1.0ms preprocess, 75.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


  1%|▏         | 1/80 [00:00<00:17,  4.55it/s]


image 1/1 /home/kroep/dev/projects/bees/bee_detection/datasets/split/val/images/20230609c318.jpg: 384x640 27 bees, 5.4ms
Speed: 1.0ms preprocess, 5.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


  2%|▎         | 2/80 [00:00<00:13,  5.80it/s]

...

 98%|█████████▊| 78/80 [00:11<00:00,  6.35it/s]


image 1/1 /home/kroep/dev/projects/bees/bee_detection/datasets/split/val/images/20230609e348.jpg: 384x640 6 bees, 4.5ms
Speed: 1.1ms preprocess, 4.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


 99%|█████████▉| 79/80 [00:11<00:00,  6.65it/s]


image 1/1 /home/kroep/dev/projects/bees/bee_detection/datasets/split/val/images/20230711c8160.jpg: 384x640 3 bees, 4.5ms
Speed: 1.1ms preprocess, 4.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████| 80/80 [00:12<00:00,  6.61it/s]
